In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Sun/ML Shock/Final project/

/content/drive/MyDrive/Sun/ML Shock/Final project


In [3]:
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from scipy.signal import spectrogram
import pickle
from tqdm import tqdm

In [4]:

# path = '../'

# folder_eeg =  'sample_data/test_eegs/'
# folder_spec =   'sample_data/test_spectrograms/'
# train = False
# df = pd.read_csv(path + 'sample_data/test.csv')

# # folder_eeg =  'sample_data/train_eegs/'
# # folder_spec =  'sample_data/train_spectrograms/'
# # train =  True #
# # df = pd.read_csv(path + 'sample_data/train.csv')



In [5]:
path = '/content/drive/MyDrive/Sun/ML Shock/Final project/'

# folder_eeg =  'data/test_eegs/'
# folder_spec =   'data/test_spectrograms/'
# train = False
# df = pd.read_csv(path + 'data/test.csv')

folder_eeg =  'data/train_eegs/' #sample_
folder_spec =  'data/train_spectrograms/'  #sample_
train =  True #
df = pd.read_csv(path + 'data/train.csv') #sample_

In [6]:

# files = os.listdir(path + folder_eeg)
# df_data = df[df['eeg_id'].isin([int(f.split('.')[0]) for f in files])]
df_data = df.copy()

if not train:
    df_data['eeg_label_offset_seconds'] = 0
    df_data['spectrogram_label_offset_seconds'] = 0

In [7]:
df_data.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [8]:
class_votes = ['seizure_vote', 'lpd_vote','gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
# classes = [v.split('_')[0] for v in class_votes]
label_cols = ['expert_consensus','label_id']


df_gr = df_data.groupby(['eeg_id', 'spectrogram_id'])
info = {}
for name, group in df_gr:
    if train:
        X = group[['eeg_label_offset_seconds', 'spectrogram_label_offset_seconds']+label_cols+class_votes]
    else:
        X = group[['eeg_id','eeg_label_offset_seconds', 'spectrogram_label_offset_seconds']]
    info[name] = X


In [9]:


def preprocessing(info, path, folder_eeg, folder_spec, train):
    freq_eeg = 200 #Hz


    # load parquet files
    data = {}
    for k in info.keys():
        d = {}
        d['eeg'] = pq.read_table(path + folder_eeg +  str(k[0])+'.parquet')
        d['spec'] = pq.read_table(path + folder_spec +  str(k[1])+'.parquet')
        d['info'] = info[k]
        data[k] = d

    spec_types = ['LL', 'RL', 'LP', 'RP']
    # select data from data_spec[0] corresponding to each type
    for k in data.keys():
        spec_data = {}
        cols = list(data[k]['spec'].to_pandas().columns)
        spec_data['spec_freq (Hz)'] = np.array([float(c.split('_')[-1]) for c in cols if c.split('_')[0] == 'LL'])
        spec_data['spec_time (s)'] = data[k]['spec'].to_pandas().filter(regex='time').values.reshape(-1)

        for spec_type in spec_types:
            spec_data[spec_type] = data[k]['spec'].to_pandas().filter(regex=spec_type).values

        data[k]['spec_sep'] = spec_data

    for k in data.keys():
        eeg_data = {}
        cols = list(data[k]['eeg'].to_pandas().columns)

        eeg_data['eeg_time (s)'] = np.arange(1, data[k]['eeg'].to_pandas().shape[0]+1)/freq_eeg

        for type in cols:
            eeg_data[type] = data[k]['eeg'].to_pandas()[type].values

        data[k]['eeg_sep'] = eeg_data

    ####
    if train:
        item_id = 'label_id'
    else:
        item_id = 'eeg_id'

    data_preprocessed = {}
    for k in data.keys():

        if train:
            votes = data[k]['info'][class_votes].to_numpy()
            votes_list = [votes[i,:] for i in range(votes.shape[0])]
            consensus = data[k]['info']['expert_consensus'].to_list()

        for kk, offset_eeg, offset_spec, i in zip(data[k]['info'][item_id].to_list(), data[k]['info']['eeg_label_offset_seconds'].to_list(), data[k]['info']['spectrogram_label_offset_seconds'].to_list(), range(len(data[k]['info']['spectrogram_label_offset_seconds'].to_list()))):

            d = {}

            for kkk in data[k]['spec_sep'].keys():
                if kkk == 'spec_freq (Hz)':
                    d[kkk] = data[k]['spec_sep'][kkk]
                elif kkk == 'spec_time (s)':
                    freq_spec = 1/(data[k]['spec_sep'][kkk][1]-data[k]['spec_sep'][kkk][0])
                    start = int((0+offset_spec)*freq_spec)
                    end = int((600+offset_spec)*freq_spec)-1
                    d[kkk] = data[k]['spec_sep'][kkk][start:end]
                else:
                    start = int((0+offset_spec)*freq_spec)
                    end = int((600+offset_spec)*freq_spec)-1
                    if start >= len(data[k]['spec_sep'][kkk]) or end >= len(data[k]['spec_sep'][kkk]):
                        print('issue spec:', start, end, data[k]['spec_sep'][kkk].shape, data[k]['spec_sep'][kkk][start:end, :].shape)
                    else:
                        d['data_spec_'+kkk] = np.log(data[k]['spec_sep'][kkk][start:end, :]+1)

            for kkk in data[k]['eeg_sep'].keys():
                start = int((0+offset_eeg)*freq_eeg)
                end = int((49+offset_eeg)*freq_eeg)
                if start >= len(data[k]['eeg_sep'][kkk]) or end >= len(data[k]['eeg_sep'][kkk]):
                    print('issue eeg:', start, end, data[k]['eeg_sep'][kkk].shape, data[k]['eeg_sep'][kkk][start:end].shape)
                else:
                    if kkk == 'eeg_time (s)':
                        d[kkk] = data[k]['eeg_sep'][kkk][start:end]
                    else:
                        d['data_eeg_'+kkk] = data[k]['eeg_sep'][kkk][start:end]

            d['freq'] = (freq_eeg, freq_spec)
            if train:
                d['class_label'] = consensus[i]
                d['class_votes'] = votes_list[i]
                d['class_probs'] = votes_list[i]/np.sum(votes_list[i])
            d['labels'] = class_votes

            del freq_spec

            data_preprocessed[kk] = d

    # first ken in data_processed
    kk = [k for k in data_preprocessed.keys()][0]
    eeg_types = [x for x in data_preprocessed[kk].keys() if 'data_eeg' in x]
    for kk in data_preprocessed.keys():
        for i, k in enumerate(eeg_types):
            _, _, Sxx = spectrogram(data_preprocessed[kk][k], fs=data_preprocessed[kk]['freq'][0])
            data_preprocessed[kk][k+'_spec'] = np.log(Sxx+1)

    return  data_preprocessed


## Run on sample

In [10]:
# data_preprocessed = preprocessing(info, path, folder_eeg, folder_spec, train)

## Check data is consistent

In [11]:
# special_keys = ['spec_freq (Hz)', 'spec_time (s)','eeg_time (s)','freq',
#                     'class_label','class_votes','class_probs','labels',]

In [12]:
# shapes = []
# freqs = []
# count = 0
# for kk in data_preprocessed.keys():
#     for k in data_preprocessed[kk].keys():
#         if k not in special_keys:
#             if data_preprocessed[kk][k].shape not in shapes:
#                 shapes.append(data_preprocessed[kk][k].shape)
#         if data_preprocessed[kk]['freq'] not in freqs:
#             freqs.append(data_preprocessed[kk]['freq'])

#     count+=1

In [13]:
# count, shapes, freqs
# train
# (69, [(299, 100), (9800,), (129, 43)], [(200, 0.5)])
# test
# (1, [(299, 100), (9800,), (129, 43)], [(200, 0.5)])

In [14]:
feature_names = ['spec_freq (Hz)', 'spec_time (s)', 'data_spec_LL', 'data_spec_RL', 'data_spec_LP', 'data_spec_RP', 'eeg_time (s)', 'data_eeg_Fp1', 'data_eeg_F3', 'data_eeg_C3', 'data_eeg_P3', 'data_eeg_F7', 'data_eeg_T3', 'data_eeg_T5', 'data_eeg_O1', 'data_eeg_Fz', 'data_eeg_Cz', 'data_eeg_Pz', 'data_eeg_Fp2', 'data_eeg_F4', 'data_eeg_C4', 'data_eeg_P4', 'data_eeg_F8', 'data_eeg_T4', 'data_eeg_T6', 'data_eeg_O2', 'data_eeg_EKG', 'freq', 'class_label', 'class_votes', 'class_probs', 'labels', 'data_eeg_Fp1_spec', 'data_eeg_F3_spec', 'data_eeg_C3_spec', 'data_eeg_P3_spec', 'data_eeg_F7_spec', 'data_eeg_T3_spec', 'data_eeg_T5_spec', 'data_eeg_O1_spec', 'data_eeg_Fz_spec', 'data_eeg_Cz_spec', 'data_eeg_Pz_spec', 'data_eeg_Fp2_spec', 'data_eeg_F4_spec', 'data_eeg_C4_spec', 'data_eeg_P4_spec', 'data_eeg_F8_spec', 'data_eeg_T4_spec', 'data_eeg_T6_spec', 'data_eeg_O2_spec', 'data_eeg_EKG_spec']

feature_eeg_spec = [f for f in feature_names if 'data_eeg' in f and '_spec' in f]
feature_eeg_raw = [f for f in feature_names if 'data_eeg' in f and '_spec' not in f]
feature_spec = [f for f in feature_names if 'data_spec_' in f]

def get_eeg_raw_features(data):
    X = np.zeros((len(data.keys()),9800, len(feature_eeg_raw), ))

    count = 0
    for k in data.keys():
        for i, f in enumerate(feature_eeg_raw):
            X[count, :,i] = data[k][f].reshape(-1)

        count +=1

    return X

def get_eeg_spec_features(data):
    X = np.zeros((len(data.keys()),129, 43, len(feature_eeg_spec),))

    count = 0
    for k in data.keys():
        for i, f in enumerate(feature_eeg_spec):
            X[count, :,:,i] = data[k][f]

        count +=1

    return X

def get_spec_features(data):
    X = np.zeros((len(data.keys()),299, 100, len(feature_spec), ))

    count = 0
    for k in data.keys():
        for i, f in enumerate(feature_spec):
            X[count, :,:,i] = data[k][f]

        count +=1

    return X

In [19]:

def save_data_to_npy(data, path_out):
    ids = [k for k in data.keys()]

    done = False

    # for id in [k for k in data.keys()]:
    #   if not os.path.exists(path_out + f'y_{str(id)}.npy'):
    #     ids.append(id)

    # if len(ids)==0:
    #   done = True


    if not done:

        file_types = ['eeg_raw', 'eeg_spec', 'spec']
        get_funcs = [get_eeg_raw_features, get_eeg_spec_features, get_spec_features]

        Xs = [[] for _ in range(len(file_types))]

        for i, file_type in enumerate(file_types):
            Xs[i].append(get_funcs[i](data))

        Xs = [np.concatenate(XX, axis=0) for XX in Xs]

        for j, id in enumerate(ids):
          np.save(path_out + f'y_{str(id)}.npy',  np.array(data[id]['class_probs']).reshape(-1))
          for i, file_type in enumerate(file_types):
              np.save(path_out + f'X_{file_type}_{str(id)}.npy', Xs[i][j,:])



## Run batched

In [ ]:
for k in tqdm(info.keys()):
  path_out = './preprocessing/'
  ids = []
  if train:
      item_id = 'label_id'
  else:
      item_id = 'eeg_id'
  for id in [x for x in info[k][item_id].to_list()]:
      if not os.path.exists(path_out + f'y_{str(id)}.npy'):
        ids.append(id)

  if len(ids)==0:
    pass
  else:
    data_preprocessed = preprocessing({k:info[k]}, path, folder_eeg, folder_spec, train)
    save_data_to_npy(data_preprocessed, path_out)




  9%|▉         | 1551/17089 [18:23<11:44:51,  2.72s/it]

In [ ]:
ids = [k for k in data_preprocessed.keys()]
y_files = [f'./preprocessing/y_{str(id)}.npy' for id in ids]
file_type = 'eeg_spec'
X_files = [f'./preprocessing/X_{file_type}_{str(id)}.npy' for id in ids]
mydata = [np.load(file, allow_pickle=True) for file in y_files]

In [ ]:
mydata[0].shape, mydata[0]